# Chapter 5. Embedding Words and Types

When implementing natural language processing tasks, we need to deal with different kinds of discrete types. The most obvious example is words. Words come from a finite set (aka vocabulary). Other examples of discrete types include characters, part-of-speech tags, named entities, named entity types, parse features, items in a product catalog, and so on. Essentially, when any input feature comes from a finite (or a countably infinite) set, it is a discrete type.

Representing discrete types (e.g., words) as dense vectors is at the core of deep learning’s successes in NLP. The terms “representation learning” and “embedding” refer to learning this mapping from one discrete type to a point in the vector space. When the discrete types are words, the dense vector representation is called a word embedding. We saw examples of count-based embedding methods, like Term-Frequency-Inverse-Document-Frequency (TF-IDF), in Chapter 2. In this chapter, we focus on learning-based or prediction-based (Baroni et al., 2014) embedding methods, in which the representations are learned by maximizing an objective for a specific learning task; for example, predicting a word based on context. Learning-based embedding methods are now de jure because of their broad applicability and performance. In fact, the ubiquity of word embeddings in NLP tasks has earned them the title of the “Sriracha of NLP,” because you can utilize word embeddings in any NLP task and expect the performance of the task to improve.1 But we contend that this sobriquet is misleading, as, unlike Sriracha, embeddings are not usually added as an afterthought to a model, but are a fundamental ingredient of the model itself.

In this chapter, we discuss vector representations as they pertain to word embeddings: methods to embed words, methods to optimize word embeddings for both supervised and unsupervised language tasks, methods to visualize word embeddings, and methods to combine word embeddings for sentences and documents. However, you must keep in mind that the methods we describe here apply to any discrete type.

## Why Learn Embeddings?
In the previous chapters, you saw traditional methods for creating vector representations of words. Specifically, you learned that you can use one-hot representations—vectors that are the same length as the size of the vocabulary and that have 0s everywhere except a single position that has the value 1 to represent a specific word. Additionally, you saw count representations—vectors that are also the same length as the number of unique words in your model but instead have counts at positions in the vector corresponding to the frequency of words in a sentence. The count-based representations are also called distributional representations because their significant content or meaning is represented by multiple dimensions in the vector. Distributional representations have a long history (Firth, 1935) and work well with many machine learning and neural network models. These representations are not learned from the data but heuristically constructed.2

Distributed representations earn their name from the fact that the words are now represented by a much lower-dimension dense vector (say d=100, as opposed to the size of the entire vocabulary, which can be around 105 to 106 or higher), and the meaning and other properties of a word are distributed across different dimensions of this dense vector.

Low-dimensional learned dense representations have several benefits over the one-hot and count-based vectors we saw in previous chapters. First, reducing the dimensionality is computationally efficient. Second, the count-based representations result in high-dimensional vectors that redundantly encode similar information along many dimensions, and do not share statistical strength. Third, very high dimensions in the input can result in real problems in machine learning and optimization—a phenomenon that’s often called the curse of dimensionality. Traditionally, to deal with this dimensionality problem, dimensionality reduction approaches like singular value decomposition (SVD) and principal component analysis (PCA) are employed, but somewhat ironically, these approaches do not scale well when dimensionality is on the order of millions (the typical case in NLP). Fourth, representations learned (or fine-tuned) from task-specific data are optimal for the task at hand. With heuristics like TF-IDF or low-dimensional approaches like SVD it is not clear if the optimization objective of the embedding approach is relevant to the task.

## Efficiency of Embeddings
To understand how embeddings work, let’s take a look at an example of a one-hot vector multiplying the weight matrix in a Linear layer, as demonstrated in Figure 5-1. In Chapters 3 and 4, the one-hot vectors were the same size as the vocabularies. The vector is called “one-hot” because it has a 1 in an index that indicates the presence of specific word.

<div align="center" style="width: 100%;">
    <div align="center" style="width: 700px">
        <img src="imgs/nlpp_0501.png">
        <h4 style="font-family: courier; font-size: .8em;">Figure 5-1. An example of matrix multiplication using a one-hot encoded vector and the weight matrix of a Linear layer. Because a one-hot vector is all 0s and a single 1, the placement of the single 1 will act as a selector in the matrix multiplication. This is visually shown using the shading patterns in the weight matrix and resultant vector. Although functional, this method of lookup is computationally expensive and inefficient because the one-hot vector is multiplying every number in the weight matrix and computing the sum for each row.</h4>
    </div>
</div>

By definition, the weight matrix of a Linear layer that accepts as input this one-hot vector must have the same number of rows as the size of the one-hot vector. When you perform the matrix multiplication, as shown in Figure 5-1, the resulting vector is actually just selecting the row indicated by the non zero entry. Based on this observation, we can just skip the multiplication step and instead directly use an integer as an index to retrieve the selected row.

One final note about the efficiency of embeddings: despite the example in Figure 5-1 showing a weight matrix that has the same dimensionality as the incoming one-hot vector, this is not always the case. In fact, embeddings are often used to represent words in a lower-dimensional space than would be needed if a one-hot vector or a count-based representation was used. Typical sizes for embeddings in the research literature range from 25 dimensions to 500 dimensions, and the exact choice can boil down to the amount of GPU memory you have to spare.

### Approaches to Learning Word Embeddings
The goal of this chapter is not to teach specific word embedding techniques, but to help you understand what embeddings are, how and where they are applicable, how to use them reliably in models, and their limitations. Our choice in doing so is because practitioners rarely find themselves in situations in which they need to write new word embedding training algorithms. However, in this section, we provide a brief overview of the current approaches to train word embeddings. All word embedding methods train with just words (i.e., unlabeled data), but in a supervised fashion. This is possible by constructing auxiliary supervised tasks in which the data is implicitly labeled, with the intuition that a representation that is optimized to solve the auxiliary task will capture many statistical and linguistic properties of the text corpus in order to be generally useful. Here are some examples of such auxiliary tasks:

* Given a sequence of words, predict the next word. This is also called the language modeling task.
* Given a sequence of words before and after, predict the missing word.
* Given a word, predict words that occur within a window, independent of the position.

Of course, this list is not complete, and the choice of the auxiliary task depends on the intuition of the algorithm designer and the computational expense. Examples include GloVe, Continuous Bag-of-Words (CBOW), Skipgrams, and so on. We refer you to Goldberg, 2017, Chapter 10, for details, but we will briefly study the CBOW model. However, for most purposes, using pretrained word embeddings and fine-tuning them for the task at hand appears sufficient.

### The Practical Use of Pretrained Word Embeddings
The bulk of this chapter, and later ones in the book, concerns itself with using pretrained word embeddings. Pretrained word embeddings trained on a large corpus—like all Google News, Wikipedia, or Common Crawl3—using one of the many methods described earlier are available freely to download and use. The rest of this chapter shows how to load and look up these embeddings efficiently, examines some properties of word embeddings, and gives some examples of using pretrained embeddings in NLP tasks.

```
dog      -1.242  -0.360  0.573  0.367  0.600  -0.189  1.273  ...
cat      -0.964  -0.610  0.674  0.351  0.413  -0.212  1.380  ...
```

To efficiently load and process embeddings, we describe a utility class called PreTrainedEmbeddings (Example 5-1). The class builds an in-memory index of all the word vectors to facilitate quick lookups and nearest-neighbor queries using an approximate nearest-neighbor package, annoy.

In [ ]:
# Example 5-1. Using pretrained word embeddings
import numpy as np
from annoy import AnnoyIndex

class PreTrainedEmbeddings(object):
    def __init__(self, word_to_index, word_vectors):
        """
        Args:
            word_to_index (dict): mapping from word to integers
            word_vectors (list of numpy arrays)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = \
            {v: k for k, v in self.word_to_index.items()}
        self.index = AnnoyIndex(len(word_vectors[0]),
                                metric='euclidean')
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        
    @classmethod
    def from_embeddings_file(cls, embedding_file):
        """Instantiate from pretrained vector file.
        
        Vector file should be of the format:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
        
        Args:
            embedding_file (str): location of the file
        Returns:
            instance of PretrainedEmbeddings
        """
        word_to_index = {}
        word_vectors = []
        with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
        return cls(word_to_index, word_vectors)

In [ ]:
%%time

embeddings = PreTrainedEmbeddings.from_embeddings_file('data/glove.6B.100d.txt')

In these examples, we use the GloVe word embeddings. After you download them, you can instantiate with the PretrainedEmbeddings class, as shown in the second input in Example 5-1.

### RELATIONSHIPS BETWEEN WORD EMBEDDINGS

The core feature of word embeddings is that the encode syntactic and semantic relationships that manifest as regularities in word use. For example, cats and dogs are talked about in very similar ways (discussions of pets, feeding, etc.). As a consequence, their embeddings are far closer to each other than they are to those of other animals, like ducks and elephants.

We can explore the semantic relationships encoded in word embeddings in several ways. One of the most popular methods is an analogy task (a popular category of reasoning tasks at exams like SAT):

```
Word1 : Word2 :: Word3 : ______
```

In this task, you are provided with the first three words and need to determine the fourth word that’s congruent to the relationship between the first two words. Using word embeddings, we can encode this spatially. First, we subtract Word2> from Word1. This difference vector encodes the relationship between Word1 and Word2. This difference can then be added to Word3 to produce a vector that’s close to the fourth word, where the blank symbol is. Doing a nearest-neighbor query on the index with this result vector solves the analogy problem. A function for computing this, shown in Example 5-2, does exactly what was just described: using vector arithmetic and the approximate nearest-neighbor index, it completes the analogy.

In [ ]:
# Example 5-2. The analogy task using word embeddings

class PreTrainedEmbeddings(object):
    """ implementation continued from previous code example"""
    def get_embedding(self, word):
        """
        Args:
            word (str)
        Returns
            an embedding (numpy.ndarray)
        """
        return self.word_vectors[self.word_to_index[word]]
    def get_closest_to_vector(self, vector, n=1):
        """Given a vector, return its n nearest neighbors
        Args:
            vector (np.ndarray): should match the size of the vectors
                in the Annoy index
            n (int): the number of neighbors to return
        Returns:
            [str, str, ...]: words nearest to the given vector
                The words are not ordered by distance
        """
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor]
                   for neighbor in nn_indices]

    def compute_and_print_analogy(self, word1, word2, word3):
        """Prints the solutions to analogies using word embeddings

        Analogies are word1 is to word2 as word3 is to __
        This method will print: word1 : word2 :: word3 : word4
        
        Args:
            word1 (str)
            word2 (str)
            word3 (str)
        """
        vec1 = self.get_embedding(word1)
        vec2 = self.get_embedding(word2)
        vec3 = self.get_embedding(word3)

        # Simple hypothesis: Analogy is a spatial relationship
        spatial_relationship = vec2 - vec1
        vec4 = vec3 + spatial_relationship

        closest_words = self.get_closest_to_vector(vec4, n=4)
        existing_words = {word1, word2, word3}
        closest_words = [word for word in closest_words 
                              if word not in existing_words] 

        if not closest_words:
            print("Could not find nearest neighbors for the vector!")
            return

        for word4 in closest_words:
            print(f"{word1} : {word2} :: {word3} : {word4}")

Interestingly, the simple word analogy task can demonstrate that word embeddings capture a variety of semantic and syntactic relationships, as demonstrated in Example 5-3.

In [ ]:
# Example 5-3. Word embeddings encode many linguistics relationships, as illustrated using the SAT analogy task
embeddings = PreTrainedEmbeddings()

In [ ]:
# Relationship 1: the relationship between gendered nouns and pronouns
embeddings.compute_and_print_analogy('man', 'he', 'woman')

In [ ]:
# Relationship 2: Verb-noun relationships
embeddings.compute_and_print_analogy('fly', 'plane', 'sail')

In [ ]:
# Relationship 3: Noun-noun relationships
embeddings.compute_and_print_analogy('cat', 'kitten', 'dog')

In [ ]:
# Relationship 4: Hypernymy (broader category)
embeddings.compute_and_print_analogy('blue', 'color', 'dog')

In [ ]:
# Relationship 5: Meronymy (part-to-whole)
embeddings.compute_and_print_analogy('toe', 'foot', 'finger')

In [ ]:
# Relationship 6: Troponymy (difference in manner)
embeddings.compute_and_print_analogy('talk', 'communicate', 'read')

In [ ]:
# Relationship 7: Metonymy (convention / figures of speech)
embeddings.compute_and_print_analogy('blue', 'democrat', 'red')

In [ ]:
# Relationship 8: Adjectival scales
embeddings.compute_and_print_analogy('fast', 'fastest', 'young')

Although it seems like the relationships are systematic to how language functions, things can get tricky. As Example 5-4 shows, because word vectors are just based on cooccurrences, relationships can be wrong.

In [ ]:
# Example 5-4. An example illustrating the danger of using cooccurrences to encode meaning—sometimes they do not!
embeddings.compute_and_print_analogy('fast', 'fastest', 'small')

Example 5-5 illustrates how one of the most common analogy pairings is encoding gendered roles.

In [ ]:
#Example 5-5. Watch out for protected attributes such as gender encoded in word embeddings. This can introduce unwanted biases in downstream models.
embeddings.compute_and_print_analogy('man', 'king', 'woman')

It turns out that differentiating between language regularities and codified cultural biases is difficult. For example, doctors are not de facto men and nurses are not de facto women, but these long-standing biases in culture are observed as the regularities in language and are codified in the word vectors, as shown in Example 5-6.

In [ ]:
# Example 5-6. Cultural gender bias encoded in vector analogy
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')

You need to be aware of the biases in embeddings, given that their popularity and use in NLP applications are on the rise. Debiasing existing word embeddings is a new and exciting research area (see Bolukbasi et al., 2016). Further, we recommend that you visit ethicsinnlp.org for latest results at the intersectionality of ethics and NLP.